In [ ]:
%matplotlib inline

Slicing {#slice_example}
=======

Extract thin planar slices from a volume.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pyvista as pv
from pyvista import examples

PyVista meshes have several slicing filters bound directly to all
datasets. These filters allow you to slice through a volumetric dataset
to extract and view sections through the volume of data.

One of the most common slicing filters used in PyVista is the
`pyvista.DataSetFilters.slice_orthogonal`{.interpreted-text role="func"}
filter which creates three orthogonal slices through the dataset
parallel to the three Cartesian planes. For example, let\'s slice
through the sample geostatistical training image volume. First, load up
the volume and preview it:


In [ ]:
mesh = examples.load_channels()
# define a categorical colormap
cmap = plt.cm.get_cmap("viridis", 4)

mesh.plot(cmap=cmap)

Note that this dataset is a 3D volume and there might be regions within
this volume that we would like to inspect. We can create slices through
the mesh to gain further insight about the internals of the volume.


In [ ]:
slices = mesh.slice_orthogonal()

slices.plot(cmap=cmap)

The orthogonal slices can be easily translated throughout the volume:


In [ ]:
slices = mesh.slice_orthogonal(x=20, y=20, z=30)
slices.plot(cmap=cmap)

We can also add just a single slice of the volume by specifying the
origin and normal of the slicing plane with the
`pyvista.DataSetFilters.slice`{.interpreted-text role="func"} filter:


In [ ]:
# Single slice - origin defaults to the center of the mesh
single_slice = mesh.slice(normal=[1, 1, 0])

p = pv.Plotter()
p.add_mesh(mesh.outline(), color="k")
p.add_mesh(single_slice, cmap=cmap)
p.show()

Adding slicing planes uniformly across an axial direction can also be
automated with the
`pyvista.DataSetFilters.slice_along_axis`{.interpreted-text role="func"}
filter:


In [ ]:
slices = mesh.slice_along_axis(n=7, axis="y")

slices.plot(cmap=cmap)

Slice Along Line
================

We can also slice a dataset along a `pyvista.Spline`{.interpreted-text
role="func"} or `pyvista.Line`{.interpreted-text role="func"} using the
`pyvista.DataSetFilters.slice_along_line`{.interpreted-text role="func"}
filter.

First, define a line source through the dataset of interest. Please note
that this type of slicing is computationally expensive and might take a
while if there are a lot of points in the line - try to keep the
resolution of the line low.


In [ ]:
model = examples.load_channels()


def path(y):
    """Equation: x = a(y-h)^2 + k"""
    a = 110.0 / 160.0**2
    x = a * y**2 + 0.0
    return x, y


x, y = path(np.arange(model.bounds[2], model.bounds[3], 15.0))
zo = np.linspace(9.0, 11.0, num=len(y))
points = np.c_[x, y, zo]
spline = pv.Spline(points, 15)
spline

Then run the filter


In [ ]:
slc = model.slice_along_line(spline)
slc

In [ ]:
p = pv.Plotter()
p.add_mesh(slc, cmap=cmap)
p.add_mesh(model.outline())
p.show(cpos=[1, -1, 1])

Multiple Slices in Vector Direction
===================================

Slice a mesh along a vector direction perpendicularly.


In [ ]:
mesh = examples.download_brain()

# Create vector
vec = np.array([1.0, 2.0, 1.0])
# Normalize the vector
normal = vec / np.linalg.norm(vec)

# Make points along that vector for the extent of your slices
a = mesh.center + normal * mesh.length / 3.0
b = mesh.center - normal * mesh.length / 3.0

# Define the line/points for the slices
n_slices = 5
line = pv.Line(a, b, n_slices)

# Generate all of the slices
slices = pv.MultiBlock()
for point in line.points:
    slices.append(mesh.slice(normal=normal, origin=point))

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh.outline(), color="k")
p.add_mesh(slices, opacity=0.75)
p.add_mesh(line, color="red", line_width=5)
p.show()

Slice At Different Bearings
===========================

From
[pyvista-support\#23](https://github.com/pyvista/pyvista-support/issues/23)

An example of how to get many slices at different bearings all centered
around a user-chosen location.

Create a point to orient slices around


In [ ]:
ranges = np.array(model.bounds).reshape(-1, 2).ptp(axis=1)
point = np.array(model.center) - ranges * 0.25

Now generate a few normal vectors to rotate a slice around the z-axis.
Use equation for circle since its about the Z-axis.


In [ ]:
increment = np.pi / 6.0
# use a container to hold all the slices
slices = pv.MultiBlock()  # treat like a dictionary/list
for theta in np.arange(0, np.pi, increment):
    normal = np.array([np.cos(theta), np.sin(theta), 0.0]).dot(np.pi / 2.0)
    name = f'Bearing: {np.rad2deg(theta):.2f}'
    slices[name] = model.slice(origin=point, normal=normal)
slices

And now display it.


In [ ]:
p = pv.Plotter()
p.add_mesh(slices, cmap=cmap)
p.add_mesh(model.outline())
p.show()